In [146]:
%use coroutines

In [9]:
// Java
import java.util.Base64
import javax.crypto.Cipher
import javax.crypto.SecretKeyFactory
import javax.crypto.spec.PBEKeySpec
import javax.crypto.spec.PBEParameterSpec

// Kotlin
import kotlin.random.Random
import kotlin.properties.Delegates

In [42]:
var str = "Hello !!!😎world @1233*@@():)👌👌👌 ] ["

// [^ ... ]  Match any character that is not in the set.
// Matches a character in the range "A" to "Z" (char code 65 to 90). Case sensitive.
// Matches a character in the range "a" to "z" (char code 97 to 122). Case sensitive.
// Matches a character in the range "0" to "9" (char code 48 to 57). Case sensitive.
// Matches a SPACE character (char code 32). 

val re = "[^A-Za-z0-9 ]".toRegex()
str = re.replace(str, "")

println(str)

Hello world 1233  


In [38]:
var c: Char

c = 'A'
while (c <= 'z') {
    print("$c ")
    ++c
}

A B C D E F G H I J K L M N O P Q R S T U V W X Y Z [ \ ] ^ _ ` a b c d e f g h i j k l m n o p q r s t u v w x y z 

In [67]:
val bytes = byteArrayOf(75, 111, 116, 108, 105, 110)

val string = String(bytes)

println(string)

Kotlin


In [141]:
val chars = ('a'..'z') + ('A'..'Z') + ('0'..'9')
fun randomID(length: Int = 64): String = List(length) { chars.random() }.joinToString("")

println(chars);
println(randomID())

[a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z, A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R, S, T, U, V, W, X, Y, Z, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
DoV8A5lP9DbJ8sQif36VKcKRbB9xlMfm9dXC2JoAW7xEcSXyCPTfsD8hBhz7RFwB


In [144]:
// https://stackoverflow.com/questions/54143361/what-is-pbewithmd5anddes
// https://gist.github.com/rohitshampur/da5f79c34260150aafc1
// https://www.bezkoder.com/kotlin-base64/

val secretCode = randomID().toCharArray()
val salt = List(8) { chars.random() }.joinToString("").toByteArray() // Salt must be 8 bytes long

fun encrypt(value: String?): String {
    val strBytes = value?.toByteArray()
    val keyFactory = SecretKeyFactory.getInstance("PBEWithMD5AndDES")
    val key = keyFactory.generateSecret(PBEKeySpec(secretCode))
    val cipher = Cipher.getInstance("PBEWithMD5AndDES").also {
        it.init(
            Cipher.ENCRYPT_MODE, key,
            PBEParameterSpec(
                salt, 20
            )
        )
    }
    // println(String(cipher.doFinal(strBytes)))
    // println()
    return Base64.getEncoder().withoutPadding().encodeToString(cipher.doFinal(strBytes))
}

fun decode(value: String?): ByteArray {
    val decodedBytes = Base64.getDecoder().decode(value)
    return decodedBytes
}

fun decrypt(value: String?): String {
    val strBytes = decode(value)
    val keyFactory = SecretKeyFactory.getInstance("PBEWithMD5AndDES")
    val key = keyFactory.generateSecret(PBEKeySpec(secretCode))
    val cipher = Cipher.getInstance("PBEWithMD5AndDES")
    cipher.init(
        Cipher.DECRYPT_MODE, key,
        PBEParameterSpec(
            salt, 20
        )
    )
    return String(cipher.doFinal(strBytes), Charsets.UTF_8)
}

val stringToEncode = randomID()
val encryptedString = encrypt(stringToEncode)
val decryptedString = decrypt(encryptedString)

println("initial value: $stringToEncode")
println()
println("salt: ${String(salt)}")
println("secret code: ${String(secretCode)}")
println()
println("encrypted: $encryptedString")
// println(String(decode(encryptedString)))
println()
println("decrypted: $decryptedString")
println()
println("is value equal: ${stringToEncode == decryptedString}")

initial value: mWYgynCAijSmi0RN9ZIjNuLwA1KaNgs4QwkbCRDtfhg8XrHVNVObE5YYfdrQkPrc

salt: RrUu3tj6
secret code: uOkN2znQ2cN28PFR5xHl6c22pwEcq76S4zWQvYbhfLM4Jl3VgLWt1dsQEzB3kZci

encrypted: PftzkjUE41Ae55gvAq2u4hXe1F6YersTwg5UwlbinkUjVLbkUQfDC4XcRhsPmknK+VcQXJCS6huXhNMbZ5ujmQOCS+1eiu2r

decrypted: mWYgynCAijSmi0RN9ZIjNuLwA1KaNgs4QwkbCRDtfhg8XrHVNVObE5YYfdrQkPrc

is value equal: true


In [167]:
import java.time.Instant

inline fun time(action: () -> Unit): Long {
    val start = Instant.now().toEpochMilli()
    action()
    return Instant.now().toEpochMilli() - start
}

suspend fun doWork() = coroutineScope {
    launch {
        val stringToEncode = randomID(64)
        val encryptedString = encrypt(stringToEncode)
        val decryptedString = decrypt(encryptedString)
        
        println("encrypted: $encryptedString")
        println()
        println("decrypted: $decryptedString")
    }
}

fun main() = runBlocking {
    val t = time {
        doWork()
    }
    println("Done $t ms")
}

main()

encrypted: XRr6b61lqN91lu6v3OKAH0VOrSTYC9KRT1J/gnNVN0OITHxTucY2xv03YFcRFjx+c/k6ELVaWSn2v8gHUxVVJzbOH+PpSQf2

decrypted: UKNcM0aoqAgaddUE7a7C7nC6Pedzb9TLFMrq82c6M5z74xdJhUZF0IBvwoYn4H2i
Done 47 ms
